In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from torch import nn, optim

In [25]:
data = 'D:\장우영\LOCALSEARCH\DA\DA\data\FAmerge_20230518_124958.csv'

merge_data = pd.read_csv(data, encoding='ANSI')
#print(weather_data)
print(merge_data.head(1))

        mmsi ship_name  ship_type   
0  440051540      D-01          0  \

                                                geom    cog  sog   
0  0101000020110F00000000002039676B41000000002DCF...  329.2  5.7  \

        insert_time  지방청           표지                일시   풍향   유향    기온    수온   
0  2023-05-11 10:10  부산청  신항유도등부표(랜비)  2023-05-11 10:10  196  246  16.4  17.0  \

     풍속   유속    기압  습도  
0  9.77  0.3  1017  82  


In [28]:
# 데이터를 교육 및 테스트 세트로 분할
X = merge_data[["mmsi","ship_name","ship_type","cog","sog","일시","풍향","기온","수온","풍속","유속","기압","습도"]]  # 입력 값 (excluding "geom")
y = merge_data["geom"]  # 출력 값 (geom)

#print(X)
#print(y)

            mmsi      ship_name  ship_type    cog   sog                일시   
0      440051540           D-01          0  329.2   5.7  2023-05-11 10:10  \
1      440300780            NaN          0  329.8   0.0  2023-05-11 10:10   
2      440208550            NaN          0  139.3   0.0  2023-05-11 10:10   
3      440275000  GBK EXPRESS 1         40  175.8   0.0  2023-05-11 10:10   
4      440414850   NO1 GEO SUNG         80    0.0   0.0  2023-05-11 10:10   
...          ...            ...        ...    ...   ...               ...   
16180  538010219  SAWASDEE VEGA         70  225.2  16.0  2023-05-11 10:20   
16181  440121850            NaN          0    0.0   0.0  2023-05-11 10:30   
16182  440110850  PILOT ORYUKDO         50  322.3   1.7  2023-05-11 10:30   
16183  440107840   NO.7 GEUMHWA         30    0.0   0.0  2023-05-11 10:30   
16184  440134620       HYUNJUNG         80  231.3   0.7  2023-05-11 10:30   

        풍향    기온    수온     풍속   유속    기압  습도  
0      196  16.4  17.0   9.7

In [29]:
#학습용, 검증용, 시험용으로 분리
train_size = int(327 * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+33], y[train_size:train_size+33]
X_test, y_test = X[train_size+33:], y[train_size+33:]

In [35]:
import torch
#Tensor 형태로 변환
import numpy as np

def make_Tensor(array):
    return torch.from_numpy(array).float()

X_train = make_Tensor(X_train.to_numpy())
y_train = make_Tensor(y_train.to_numpy())
X_val = make_Tensor(X_val.to_numpy())
y_val = make_Tensor(y_val.to_numpy())
X_test = make_Tensor(X_test.to_numpy())
y_test = make_Tensor(y_test.to_numpy())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_268\3470333947.py", line 8, in <module>
    X_train = make_Tensor(X_train.to_numpy())
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_268\3470333947.py", line 6, in make_Tensor
    return torch.from_numpy(array).float()
TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\SW\AppData\Roaming\Python\Pyth

In [ ]:
class RoutePrediction(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers):
        super(RoutePrediction, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.c1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size = 2, stride = 1) # 1D CNN 레이어 추가
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)